# 🏆 The SQLAlchemy 2.0 Pro Masterclass

This notebook covers the full spectrum of database engineering with SQLAlchemy, from basic schema definition to advanced performance optimization.

### Topics Covered:
1. **Modern Schema Definition** (Mapped/mapped_column)
2. **The Session Lifecycle** (Unit of Work pattern)
3. **Relationship Loading Strategies** (Avoid the N+1 problem)
4. **Hybrid Properties** (Python logic in SQL)
5. **Advanced Aggregations & Joins**
6. **Integration with Analytics** (Pandas)

In [1]:
# 1. Setup
%pip install -q sqlalchemy pandas matplotlib

## 2. Advanced Schema Design
We will implement a Store system with **Users**, **Orders**, and **Products**.



In [2]:
from datetime import datetime
from typing import List, Optional
from sqlalchemy import ForeignKey, String, select, func, Index, Float
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, relationship, Session, aliased
from sqlalchemy.ext.hybrid import hybrid_property

class Base(DeclarativeBase):
    pass

class User(Base):
    __tablename__ = "user_account"

    id: Mapped[int] = mapped_column(primary_key=True)
    username: Mapped[str] = mapped_column(String(50), unique=True, index=True)
    email: Mapped[str]

    orders: Mapped[List["Order"]] = relationship(back_populates="customer")

class Order(Base):
    __tablename__ = "order"

    id: Mapped[int] = mapped_column(primary_key=True)
    order_date: Mapped[datetime] = mapped_column(default=func.now())
    user_id: Mapped[int] = mapped_column(ForeignKey("user_account.id"))
    total_price: Mapped[float] = mapped_column(Float, default=0.0)
    status: Mapped[str] = mapped_column(String(20), default="pending")

    customer: Mapped["User"] = relationship(back_populates="orders")
    items: Mapped[List["OrderItem"]] = relationship()

    # 🧠 Hybrid Property: Logic that works in Python AND SQL
    @hybrid_property
    def is_expensive(self):
        return self.total_price > 100.0

class OrderItem(Base):
    __tablename__ = "order_item"
    id: Mapped[int] = mapped_column(primary_key=True)
    order_id: Mapped[int] = mapped_column(ForeignKey("order.id"))
    product_name: Mapped[str]
    price: Mapped[float]

print("Advanced Models with Hybrid Properties Ready.")

Advanced Models with Hybrid Properties Ready.


## 3. The Unit of Work & Performance Loading
One major trap in ORMs is the **N+1 problem** (firing 100 queries to get items for 100 orders). We solve this using `selectinload` or `joinedload`.



In [3]:
from sqlalchemy import create_engine
from sqlalchemy.orm import selectinload

engine = create_engine("sqlite://") # In-memory for speed
Base.metadata.create_all(engine)

with Session(engine) as session:
    # Seed Data
    u1 = User(username="jdoe", email="j@example.com")
    o1 = Order(total_price=150.50, customer=u1, items=[
        OrderItem(product_name="Laptop", price=140.0),
        OrderItem(product_name="Mouse", price=10.5)
    ])
    o2 = Order(total_price=45.00, customer=u1, items=[
        OrderItem(product_name="Cables", price=45.0)
    ])

    session.add_all([u1, o1, o2])
    session.commit()

    # 🚀 EAGER LOADING: Fetch orders and their items in just 2 optimized queries
    stmt = select(Order).options(selectinload(Order.items))
    orders = session.scalars(stmt).all()

    for o in orders:
        print(f"Order #{o.id} | Total: ${o.total_price} | Expensive: {o.is_expensive}")
        for item in o.items:
            print(f"  - {item.product_name}: ${item.price}")

Order #1 | Total: $150.5 | Expensive: True
  - Laptop: $140.0
  - Mouse: $10.5
Order #2 | Total: $45.0 | Expensive: False
  - Cables: $45.0


## 4. Advanced Aggregations (GROUP BY & HAVING)
Let's find the total spend per user using SQL functions inside the ORM.

In [4]:
with Session(engine) as session:
    # Query: User, Total Spent, and Order Count
    stmt = (
        select(
            User.username,
            func.sum(Order.total_price).label("total_spend"),
            func.count(Order.id).label("order_count")
        )
        .join(Order)
        .group_by(User.username)
        .having(func.sum(Order.total_price) > 50)
    )

    results = session.execute(stmt).all()
    for row in results:
        print(f"{row.username} spent ${row.total_spend:.2f} across {row.order_count} orders")

jdoe spent $195.50 across 2 orders


## 5. Raw SQL Fallback & Pandas
Sometimes you just need raw SQL. SQLAlchemy allows you to drop down to text-based queries while still using the engine connection.

In [5]:
import pandas as pd
from sqlalchemy import text

with engine.connect() as conn:
    # Using raw SQL for complex reporting
    sql_text = text("SELECT product_name, price FROM order_item WHERE price > :p")
    df = pd.read_sql_query(sql_text, conn, params={"p": 20.0})

print("--- Data Extracted to Pandas ---")
print(df)

--- Data Extracted to Pandas ---
  product_name  price
0       Laptop  140.0
1       Cables   45.0


## 6. Constraints & Safety
Learn to add database-level safeguards like Indexes and Checks.

In [6]:
from sqlalchemy import CheckConstraint

class Product(Base):
    __tablename__ = "product"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str]
    stock: Mapped[int]

    # Ensure stock never goes below zero at the DB level
    __table_args__ = (
        CheckConstraint("stock >= 0", name="check_stock_positive"),
        Index("idx_product_name", "name"),
    )

print("Safeguards added to Schema.")

Safeguards added to Schema.


In [8]:
%pip install "psycopg[binary]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 kB 14.6 MB/s eta 0:00:00


In [9]:
# 7. REAL-WORLD DEPLOYMENT: Secrets & Environment Management
# This cell demonstrates how to securely manage database credentials
# using environment variables and transition from SQLite to PostgreSQL.

import os
from sqlalchemy import create_engine
from sqlalchemy.engine import URL

# 1. SETUP ENVIRONMENT (In a real app, these are in a .env file)
# Using os.environ allows your code to run on Heroku, AWS, or Docker easily.
os.environ["DB_TYPE"] = "postgresql"  # Change to 'sqlite' for local testing
os.environ["DB_USER"] = "production_admin"
os.environ["DB_PASS"] = "super_secret_password"
os.environ["DB_HOST"] = "db.example.com"
os.environ["DB_NAME"] = "analytics_prod"

def get_production_engine():
    """
    Constructs a SQLAlchemy engine based on environment variables.
    Prevents hardcoding sensitive credentials in your source code.
    """
    db_type = os.getenv("DB_TYPE", "sqlite")

    if db_type == "sqlite":
        # Local development fallback
        return create_engine("sqlite:///dev_database.db")

    # 2. BUILDING THE CONNECTION STRING (The Secure Way)
    # The URL object handles special characters in passwords automatically
    connection_url = URL.create(
        drivername="postgresql+psycopg", # Using psycopg3 driver
        username=os.getenv("DB_USER"),
        password=os.getenv("DB_PASS"),
        host=os.getenv("DB_HOST"),
        database=os.getenv("DB_NAME"),
        port=5432
    )

    # 3. ENGINE CONFIGURATION FOR PRODUCTION
    # pool_size: number of permanent connections to keep open
    # max_overflow: extra connections allowed during traffic spikes
    # pool_recycle: prevents 'Connection Timed Out' by refreshing connections
    return create_engine(
        connection_url,
        pool_size=10,
        max_overflow=20,
        pool_recycle=3600,
        echo=False # Set to True only for debugging
    )

try:
    engine = get_production_engine()
    print(f"🚀 Engine configured for: {engine.url.drivername}")
    print("✅ Ready for secure production deployment.")
except Exception as e:
    print(f"❌ Configuration error: {e}")

# FINAL PRODUCTIVITY TIP:
# Always use 'Alembic' (pip install alembic) alongside this setup
# to manage your 'Base.metadata' changes over time!

🚀 Engine configured for: postgresql+psycopg
✅ Ready for secure production deployment.
